# Scaling ML workflows with Ray

We are going to scale an end to end machine learning workload:

1. Data loading
2. Training
2. Hyperparameter tuning
3. Inference

First, we try on a local node with a data set, time it, and then try on an Anyscale cluster with multiple nodes and multiple cores.

We should observe noticeable difference.

<img src="https://images.ctfassets.net/xjan103pcp94/ETLioF3e6PLPYr5DggvUl/7f8247e1bf79ab49295882259bd5420d/localMachineCloud.png" width="40%" height="30%">

In [1]:
import ray
import numpy as np
import os
import pandas as pd
import glob

from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split

import time
import tqdm
import xgboost as xgb
import xgboost_ray
from xgboost.callback import TrainingCallback

## Read the data files into various chunks

Read the classifation data features and labes into 300MB, 3GB, and 11GB chunks to
illustrate training, tunning, and inferencing at scale.

The data is generated from `sklearn.datasets make_classification` with 2 classes as labels as default. More rows or classes can be easily generated. For this demo, we
will use the following: 

 * 10,000,000 rows
 * 40 feature columns
 * 2 classes

In [2]:

def get_parquet_files(path, size=10):
    """Get all parquet parts from a directory."""
    size *= 10
    files = sorted(glob.glob(path))
    while size > len(files):
        files = files + files
    files = files[0:size]
    return files

def load_parquet_dataset(files):
    """Load all parquet files into a pandas df."""
    df = pd.read_parquet(files[0])
    for i in tqdm.tqdm(range(1, len(files), 50)):
        df = pd.concat((df, pd.read_parquet(files[i:i+50])))
        memory_usage = df.memory_usage(deep=True).sum()/1e9
        tqdm.tqdm.write(f"Dataset size: {memory_usage} GB")
        if memory_usage > 12:
            raise MemoryError(f"Dataset too big to fit into memory!")
    return df[sorted(df.columns)].drop("partition", axis=1)

class TqdmCallback(TrainingCallback):
    """Simple callback to print a progress bar"""
    def __init__(self, num_samples: int) -> None:
        self.num_samples = num_samples
        super().__init__()

    def before_training(self, model):
        if xgb.rabit.get_rank() == 0:
            self.pbar = tqdm.tqdm(total=self.num_samples)
        return model

    def after_iteration(self, model, epoch, evals_log):
        if xgb.rabit.get_rank() == 0:
            self.pbar.update(1)

    def after_training(self, model):
        if xgb.rabit.get_rank() == 0:
            self.pbar.close()
        return model

data_path = f"/home/ec2-user/data/classification.parquet/**/*.parquet"

data_files_300MB = get_parquet_files(data_path, size=1)
data_files_3GB = get_parquet_files(data_path, size=10)
data_files_11GB = get_parquet_files(data_path, size=30)

# 1a. Training (regular XGBoost)

In [3]:
# XGBoost config.
xgboost_params = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
}

from xgboost import DMatrix, train

def train_xgboost(config, files, progress_bar=True):
    start_time = time.time()

    target_column = "labels"

    # local loading of a parquet dataset
    # needs conversion to pandas as vanilla
    # xgboost can't work with parquet directly
    train_df = load_parquet_dataset(files[:int(len(files) * 0.75)])
    test_df = load_parquet_dataset(files[int(len(files) * 0.75):])
    train_x = train_df.drop(target_column, axis=1)
    train_y = train_df[target_column]
    test_x = test_df.drop(target_column, axis=1)
    test_y = test_df[target_column]

    train_set = DMatrix(train_x, train_y)
    test_set = DMatrix(test_x, test_y)

    evals_result = {}

    # Train the classifier
    bst = train(params=config,
                dtrain=train_set,
                evals=[(test_set, "eval")],
                evals_result=evals_result,
                verbose_eval=False,
                num_boost_round=10,
                callbacks=[TqdmCallback(10)] if progress_bar else [])
    print(f"Total time taken: {time.time()-start_time}")

    model_path = "model.xgb"
    bst.save_model(model_path)
    print("Final validation error: {:.4f}".format(
        evals_result["eval"]["error"][-1]))

    return bst

Try a smaller data set (300MB) to ensure the our xgboost trainer works

In [4]:
bst = train_xgboost(xgboost_params, data_files_300MB)

100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


Dataset size: 0.23590022 GB


100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


Dataset size: 0.101100124 GB


100%|██████████| 10/10 [00:16<00:00,  1.63s/it]

Total time taken: 18.82234835624695
Final validation error: 0.1478


Try a larger data set (3GB) to ensure the our xgboost trainer works

In [6]:
bst = train_xgboost(xgboost_params, data_files_3GB)

 50%|█████     | 1/2 [00:13<00:13, 13.10s/it]

Dataset size: 1.718702504 GB


100%|██████████| 2/2 [00:21<00:00, 10.71s/it]


Dataset size: 2.7896 GB


100%|██████████| 1/1 [00:06<00:00,  6.97s/it]


Dataset size: 0.842500748 GB


100%|██████████| 10/10 [03:41<00:00, 22.15s/it]


Total time taken: 254.95771074295044
Final validation error: 0.1472


# 1b. Training (XGBoost on Ray)

Requires only few lines of code changes to XGBoost to use distributed training on Ray:

 * `from xgboost_ray import RayDMatrix, train, RayParams`
 * Use `RayDMatrix` distributed [sharding and reading](https://github.com/ray-project/xgboost_ray#distributed-data-loading) to convert to XGBoost internal data structure
 * Add additional `RayParams` argument to `train_xgboost(...)` for level of parallelism
 
 This diagram depicts how Distributed XGBoost-Ray works on a Ray cluster
 
 <img src="images/xgboost_distributed.png" width="40%" height="30%">

Let's start Ray on the local host.

In [5]:

ray.init()

{'node_ip_address': '10.0.0.231',
 'raylet_ip_address': '10.0.0.231',
 'redis_address': '10.0.0.231:6379',
 'object_store_address': '/tmp/ray/session_2022-01-26_19-15-54_415773_15641/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-01-26_19-15-54_415773_15641/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-01-26_19-15-54_415773_15641',
 'metrics_export_port': 63950,
 'node_id': '7e2bd7cfb7e67522dcadb22a0c0e8b55d02f0a3e3ae71f32ef5de8c9'}

In [6]:
# XGBoost config.
xgboost_params = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
}

from xgboost_ray import RayDMatrix, train, RayParams

def train_xgboost_remote(config, files, ray_params, progress_bar=True):
    start_time = time.time()

    target_column = "labels"
    ds = ray.data.read_parquet(files)
    ds.repartition(32)

    split_index = int(ds.count() * (1 -0.3))
    X = ds.random_shuffle()
    X_train, X_valid = X.split_at_indices([split_index]) 

    ds_train = X_train.repartition(8)
    ds_valid = X_valid.repartition(8)
    train_set = RayDMatrix(ds_train, target_column, ignore=["partition"])
    test_set = RayDMatrix(ds_valid, target_column, ignore=["partition"])

    evals_result = {}

    # Train the classifier
    bst = train(params=config,
                dtrain=train_set,
                evals=[(test_set, "eval")],
                evals_result=evals_result,
                verbose_eval=False,
                num_boost_round=10,                       # equivalent to epochs or iterations
                callbacks=[TqdmCallback(10)] if progress_bar else [],
                ray_params=ray_params)                    # Ray parameters for parallelism
    print(f"Total time taken: {time.time()-start_time}")

    model_path = "model.xgb"
    bst.save_model(model_path)
    print("Final validation error: {:.4f}".format(
        evals_result["eval"]["error"][-1]))

    return bst

#### Make dataset available in cloud object storage
 * We uploaded the dataset in Cloud storage (S3) and changed the URL for each file in our 3 datasets. 

In [7]:
s3_data_files_300MB = [str(i).replace('/home/ec2-user/', 's3://anyscale-demo/') for i in data_files_300MB]
s3_data_files_3GB = [str(i).replace('/home/ec2-user/', 's3://anyscale-demo/') for i in data_files_3GB]
s3_data_files_11GB = [str(i).replace('/home/ec2-user/', 's3://anyscale-demo/') for i in data_files_11GB]

In [8]:
s3_data_files_300MB

['s3://anyscale-demo/data/classification.parquet/partition=0/part_0.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=1/part_1.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=10/part_10.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=11/part_11.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=12/part_12.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=13/part_13.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=14/part_14.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=15/part_15.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=16/part_16.parquet',
 's3://anyscale-demo/data/classification.parquet/partition=17/part_17.parquet']

#### Define RayParams for XGBoost level of parallelism
 * Eight actors, 
 * Each using 2-4 CPUs

In [9]:
remote_train_xgboost_remote = ray.remote(train_xgboost_remote)
bst = ray.get(remote_train_xgboost_remote.remote(xgboost_params, s3_data_files_300MB, RayParams(num_actors=4, cpus_per_actor=1)))

Repartition: 100%|██████████| 8/8 [00:00<00:00, 239.14it/s]
(train_xgboost_remote pid=15864) 2022-01-26 19:18:28,931	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(train_xgboost_remote pid=15864) 2022-01-26 19:18:31,033	INFO main.py:1024 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=15861) [19:18:31] task [xgboost.ray]:140666353827072 got new rank 2
(_RemoteRayXGBoostActor pid=15865) [19:18:31] task [xgboost.ray]:140427781286496 got new rank 3
(_RemoteRayXGBoostActor pid=15860) [19:18:31] task [xgboost.ray]:140500780257344 got new rank 1
(_RemoteRayXGBoostActor pid=16120) [19:18:31] task [xgboost.ray]:140386172489344 got new rank 0
 90%|█████████ | 9/10 [00:13<00:01,  1.32s/it]m 


(train_xgboost_remote pid=15864) Total time taken: 47.01284575462341
(train_xgboost_remote pid=15864) Final validation error: 0.1511


100%|██████████| 10/10 [00:14<00:00,  1.46s/it] 
(train_xgboost_remote pid=15864) 2022-01-26 19:18:46,298	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=700,000 in 17.43 seconds (15.26 pure XGBoost training time).


Let's now point our training job jto an anyscale cluster.

In [10]:
ray.shutdown()
ray.init("anyscale://xgboost-demo2")

Authenticating
Loaded Anyscale authentication token from ~/.anyscale/credentials.json.

Output
(anyscale +1.1s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.7s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.9s) Using project (name: phi-demos, project_dir: /home/ec2-user/working_dir, id: prj_26wrN4CCEw3fGhXMxzRAwvuk).
(anyscale +2.0s) cluster xgboost-demo2 is currently running, the cluster will not be restarted.


2022-01-26 19:19:24,408	INFO packaging.py:352 -- Creating a file package for local directory '/home/ec2-user/working_dir'.
2022-01-26 19:19:24,427	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_7954c209209536de.zip' (0.78MiB) to Ray cluster...
2022-01-26 19:19:24,899	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_7954c209209536de.zip'.


(anyscale +14.1s) Connected to xgboost-demo2, see: https://console.anyscale.com/projects/prj_26wrN4CCEw3fGhXMxzRAwvuk/clusters/ses_aqKYRsRQpAUFD4gcfPrxnZ8e
(anyscale +14.1s) URL for head node of cluster: https://session-aqkyrsrqpaufd4gcfprxnz8e.i.anyscaleuserdata.com


AnyscaleClientContext(dashboard_url='https://session-aqkyrsrqpaufd4gcfprxnz8e.i.anyscaleuserdata.com/auth/?token=af19b5ad-165f-4d57-a596-3bd9d882b96e&redirect_to=dashboard', python_version='3.8.5', ray_version='1.9.1', ray_commit='2cdbf974ea63caf4323aacbccaef2394a14a8562', protocol_version='2021-09-22', _num_clients=1, _context_to_restore=None)

We can now run our training job using our 11GB training files

In [11]:

bst = ray.get(remote_train_xgboost_remote.remote(xgboost_params, s3_data_files_11GB, RayParams(num_actors=8, cpus_per_actor=2)))

Repartition: 100%|██████████| 8/8 [00:02<00:00,  2.92it/s]
(train_xgboost_remote pid=45381) 2022-01-26 11:25:13,839	INFO main.py:979 -- [RayXGBoost] Created 8 new actors (8 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=5299, ip=10.0.0.49) [11:25:19] task [xgboost.ray]:139948544085536 got new rank 5
(_RemoteRayXGBoostActor pid=3564, ip=10.0.0.204) [11:25:19] task [xgboost.ray]:140037885963040 got new rank 2
(_RemoteRayXGBoostActor pid=3947, ip=10.0.0.152) [11:25:19] task [xgboost.ray]:140310149614992 got new rank 0
(_RemoteRayXGBoostActor pid=3500, ip=10.0.0.187) [11:25:19] task [xgboost.ray]:140020789969056 got new rank 1
(_RemoteRayXGBoostActor pid=5216, ip=10.0.0.61) [11:25:19] task [xgboost.ray]:140650302468208 got new rank 6
(_RemoteRayXGBoostActor pid=3625, ip=10.0.0.218) [11:25:19] task [xgboost.ray]:139790166167408 got new rank 3
(_RemoteRayXGBoostActor pid=7009, ip=10.0.0.85) [11:25:19] task [xgboost.ray]:140327173319840 got new rank 7


(train_xgboost_remote pid=45381) Total time taken: 523.420551776886
(train_xgboost_remote pid=45381) Final validation error: 0.1488


# 2. Hyperparameter Tuning

[Ray Tune](https://docs.ray.io/en/latest/tune/index.html) will launch distributed HPO, using 8 remote actors, each with its own instance of the trainable func: `train_xgboost` defined above.

<img src="https://raw.githubusercontent.com/dmatrix/ray-core-tutorial/main/images/ray_tune_dist_hpo.png" width="40%" height="30%">

# Weight and Biases integration
We just added the WandB callback to log all of our experiment metrics into WandB in the tune.run callback.

In [17]:
from ray import tune
import os
from ray.tune.integration.wandb import WandbLoggerCallback

project_name = f"XGBoost-Tune-Experiment-demo"
api_key = "xxxxxxxxxxxx" # TODO: change this if you have your own API key
os.environ["TUNE_DISABLE_AUTO_CALLBACK_LOGGERS"] = "1"

# Set XGBoost config.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

ray_params = RayParams(
    max_actor_restarts=1,
    gpus_per_actor=0,
    cpus_per_actor=2,
    num_actors=8)

analysis = tune.run(
    tune.with_parameters(train_xgboost_remote, files=s3_data_files_300MB, ray_params=ray_params, progress_bar=False),
    # Use the `get_tune_resources` helper function to set the resources.
    resources_per_trial=ray_params.get_tune_resources(),
    config=config,
    num_samples=8,
    metric="eval-error",
    mode="min",
    verbose=1,
    callbacks=[WandbLoggerCallback(
            project= project_name,
            api_key= api_key,
            log_config=True)]
    )

accuracy = 1. - analysis.best_result["eval-error"]
print(f"Best model parameters: {analysis.best_config}")
print(f"Best model total accuracy: {accuracy:.4f}")
print(analysis.best_config)

(run pid=5676) 2022-01-25 08:04:58,073	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(run pid=5676) 2022-01-25 08:08:26,534	WARNING trial_runner.py:276 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (149 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.


(run pid=5676) == Status ==
(run pid=5676) Current time: 2022-01-25 08:08:26 (running for 00:00:00.39)
(run pid=5676) Memory usage on this node: 12.1/30.9 GiB
(run pid=5676) Using FIFO scheduling algorithm.
(run pid=5676) Resources requested: 17.0/136 CPUs, 0/0 GPUs, 0.0/376.72 GiB heap, 0.0/156.92 GiB objects
(run pid=5676) Result logdir: /home/ray/ray_results/train_xgboost_remote_2022-01-25_08-04-58
(run pid=5676) Number of trials: 8/8 (7 PENDING, 1 RUNNING)
(run pid=5676) 
(run pid=5676) 


(run pid=5676) wandb: Currently logged in as: sublimotion (use `wandb login --relogin` to force relogin)
(run pid=5676) wandb: wandb version 0.12.9 is available!  To upgrade, please run:
(run pid=5676) wandb:  $ pip install wandb --upgrade
(run pid=5676) wandb: Currently logged in as: sublimotion (use `wandb login --relogin` to force relogin)
(run pid=5676) wandb: WARNING Tried to auto resume run with id 8b1d5_00000 but id 8b1d5_00001 is set.
(run pid=5676) wandb: Currently logged in as: sublimotion (use `wandb login --relogin` to force relogin)
(run pid=5676) wandb: WARNING Tried to auto resume run with id 8b1d5_00001 but id 8b1d5_00002 is set.
(run pid=5676) wandb: Currently logged in as: sublimotion (use `wandb login --relogin` to force relogin)
(run pid=5676) wandb: WARNING Tried to auto resume run with id 8b1d5_00002 but id 8b1d5_00003 is set.
(run pid=5676) wandb: wandb version 0.12.9 is available!  To upgrade, please run:
(run pid=5676) wandb:  $ pip install wandb --upgrade
(run

(run pid=5676) == Status ==
(run pid=5676) Current time: 2022-01-25 08:08:31 (running for 00:00:05.25)
(run pid=5676) Memory usage on this node: 12.5/30.9 GiB
(run pid=5676) Using FIFO scheduling algorithm.
(run pid=5676) Resources requested: 102.0/136 CPUs, 0/0 GPUs, 0.0/376.72 GiB heap, 0.0/156.92 GiB objects
(run pid=5676) Result logdir: /home/ray/ray_results/train_xgboost_remote_2022-01-25_08-04-58
(run pid=5676) Number of trials: 8/8 (2 PENDING, 6 RUNNING)
(run pid=5676) 
(run pid=5676) 
(run pid=5676) 


(run pid=5676) E0125 08:08:32.348522237   20921 chttp2_transport.cc:1103]   Received a GOAWAY with error code ENHANCE_YOUR_CALM and debug data equal to "too_many_pings"
(run pid=5676) wandb: Tracking run with wandb version 0.12.5
(run pid=5676) wandb: Syncing run train_xgboost_remote_8b1d5_00000
(run pid=5676) wandb:  View project at https://wandb.ai/sublimotion/XGBoost-Tune-Experiment-demo
(run pid=5676) wandb:  View run at https://wandb.ai/sublimotion/XGBoost-Tune-Experiment-demo/runs/8b1d5_00000
(run pid=5676) wandb: Run data is saved locally in /tmp/ray/session_2022-01-25_07-11-21_216444_163/runtime_resources/working_dir_files/_ray_pkg_226d4ea0735c6585/wandb/run-20220125_080827-8b1d5_00000
(run pid=5676) wandb: Run `wandb offline` to turn off syncing.


...

(run pid=5676) wandb: Waiting for W&B process to finish, PID 22358... (success).
(run pid=5676) wandb: - 0.00MB of 0.00MB uploaded (0.00MB deduped)
(run pid=5676) wandb: \ 0.00MB of 0.00MB uploaded (0.00MB deduped)
(run pid=5676) wandb: | 0.00MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: / 0.00MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: - 0.02MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: \ 0.02MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: | 0.02MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: / 0.02MB of 0.02MB uploaded (0.00MB deduped)
(run pid=5676) wandb: - 0.02MB of 0.02MB uploaded (0.00MB deduped)
wandb:                                                                                


(run pid=5676) == Status ==
(run pid=5676) Current time: 2022-01-25 08:10:58 (running for 00:02:31.86)
(run pid=5676) Memory usage on this node: 12.6/30.9 GiB
(run pid=5676) Using FIFO scheduling algorithm.
(run pid=5676) Resources requested: 0/136 CPUs, 0/0 GPUs, 0.0/376.72 GiB heap, 0.0/156.92 GiB objects
(run pid=5676) Current best trial: 8b1d5_00006 with eval-error=0.16095 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.04340709367317894, 'subsample': 0.8957024019504654, 'max_depth': 8, 'nthread': 2, 'n_jobs': 2}
(run pid=5676) Result logdir: /home/ray/ray_results/train_xgboost_remote_2022-01-25_08-04-58
(run pid=5676) Number of trials: 8/8 (8 TERMINATED)
(run pid=5676) 
(run pid=5676) 
(run pid=5676) == Status ==
(run pid=5676) Current time: 2022-01-25 08:10:58 (running for 00:02:31.91)
(run pid=5676) Memory usage on this node: 12.6/30.9 GiB
(run pid=5676) Using FIFO scheduling algorithm.
(run pid=5676) Resourc

(run pid=5676) wandb: Run history:
(run pid=5676) wandb:                 config/eta ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:           config/max_depth ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:              config/n_jobs ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:             config/nthread ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:           config/subsample ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:                 eval-error █▅▅▄▃▃▂▂▁▁
(run pid=5676) wandb:               eval-logloss █▇▆▅▄▄▃▂▂▁
(run pid=5676) wandb:   iterations_since_restore ▁▂▃▃▄▅▆▆▇█
(run pid=5676) wandb:         time_since_restore ▁▃▃▃▃▃████
(run pid=5676) wandb:           time_this_iter_s █▂▁▁▁▁▄▁▁▁
(run pid=5676) wandb:               time_total_s ▁▃▃▃▃▃████
(run pid=5676) wandb:                  timestamp ▁▃▃▃▃▃████
(run pid=5676) wandb:    timesteps_since_restore ▁▁▁▁▁▁▁▁▁▁
(run pid=5676) wandb:         training_iteration ▁▂▃▃▄▅▆▆▇█
(run pid=5676) wandb: 
(run pid=5676) wandb: Run summary:
(run pid=5676) wandb:                 config/eta 0.04341
(run pid=5

Best model parameters: {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.04340709367317894, 'subsample': 0.8957024019504654, 'max_depth': 8}
Best model total accuracy: 0.8390
{'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.04340709367317894, 'subsample': 0.8957024019504654, 'max_depth': 8}


(run pid=5676) --- Logging error ---
(run pid=5676) Traceback (most recent call last):
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/logging/__init__.py", line 1085, in emit
(run pid=5676)     self.flush()
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/logging/__init__.py", line 1065, in flush
(run pid=5676)     self.stream.flush()
(run pid=5676) OSError: [Errno 28] No space left on device
(run pid=5676) Call stack:
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/threading.py", line 890, in _bootstrap
(run pid=5676)     self._bootstrap_inner()
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
(run pid=5676)     self.run()
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 52, in run
(run pid=5676)     self._run()
(run pid=5676)   File "/home/ray/anaconda3/lib/python3.8/site-packages/wandb/sdk/internal/internal_util.py", line 103, in _run
(run 

# 3a. Inference (regular XGBoost)

In [19]:
total_time = time.time()

df = load_parquet_dataset(data_files_300MB).drop(["labels", "partition"], axis=1, errors="ignore")
inference_df = DMatrix(df)
results = bst.predict(inference_df)

print(f"Total time taken: {time.time()-total_time}")

100%|██████████| 1/1 [00:00<00:00,  1.31it/s]


Dataset size: 0.337000372 GB
Total time taken: 1.5613970756530762


In [20]:
results

array([0.7038215 , 0.18560845, 0.17597654, ..., 0.18545696, 0.87184376,
       0.30593213], dtype=float32)

# 3b. Inference (XGBoost on Ray)


In [12]:
@ray.remote
def xgb_batch_inference(bst, files):
    total_time = time.time()

    inference_df = RayDMatrix(files, ignore=["labels", "partition"])
    results = xgboost_ray.predict(bst, inference_df, ray_params=RayParams(num_actors=8))

    print(f"Total time taken: {time.time()-total_time}")
    return results

In [13]:
batch_results = ray.get(xgb_batch_inference.remote(bst, s3_data_files_300MB))


(xgb_batch_inference pid=46848) 2022-01-26 11:30:40,396	INFO main.py:1543 -- [RayXGBoost] Created 8 remote actors.
(xgb_batch_inference pid=46848) 2022-01-26 11:30:53,481	INFO main.py:1560 -- [RayXGBoost] Starting XGBoost prediction.


(xgb_batch_inference pid=46848) Total time taken: 13.341913938522339


In [14]:
batch_results

array([0.6322733 , 0.70809305, 0.36546   , ..., 0.17210634, 0.90754145,
       0.25357166], dtype=float32)

---

# References
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)

